# Fotmob Seasonwide Shotdata Scraper


This Scraper works using Selenium so make sure to download Chromedrivers (reccomend to use chrome).<br>
Link to download: https://chromedriver.chromium.org/downloads ( before download do check your Chrome version)
<br>
Note: This scraper works for Current season data for All T5 Leagues & Champions League. For Previous seasons data 
it will be eventually added & code will see modfications, until then enjoy latest Shots data with x,y coordinates & xG,xGOT values!!

In [32]:
#Import Libraries
import pandas as pd
import mplsoccer as mps
import pickle
import matplotlib.pyplot as plt
from mplsoccer import add_image
from PIL import Image
from matplotlib.colors import to_rgba
from urllib.request import urlopen
import time
import time
import pandas as pd
pd.options.mode.chained_assignment = None
import json
from bs4 import BeautifulSoup as soup
import re 
from collections import OrderedDict
import datetime
from datetime import datetime as dt
import itertools
import numpy as np
import pickle
try:
    from tqdm import trange
except ModuleNotFoundError:
    pass

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import difflib
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [33]:
pd.set_option("display.max_columns",300)
pd.set_option("display.max_rows",500)

In [28]:
# T5 League URLS
Premier_League='https://www.fotmob.com/leagues/47/overview/premier-league'
Serie_A='https://www.fotmob.com/leagues/55/overview/serie-a'
LaLiga='https://www.fotmob.com/leagues/87/overview/laliga'
Bundesliga='https://www.fotmob.com/leagues/54/overview/bundesliga'
Ligue1='https://www.fotmob.com/leagues/53/overview/ligue-1'
ChampionsLeague='https://www.fotmob.com/leagues/42/matches/champions-league'

In [34]:
# Returns a Dataframe containing Home_team,away_team, Matchid & Fotmob Match URL of the game
# Do remember to enter your driver path from your PC
driver_path='C:/Users/Rahul/Pictures/Juve Event Data/chromedriver.exe'
def FOTMOB_Leaguegames_Urls(main_url):
    list1=[]
    list2=[]
    yes=True
    # Enter Your own driver path below before you execute this function
    driver = webdriver.Chrome(driver_path) 
    driver.maximize_window()
    driver.get(main_url)
    try:
        driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div[2]/button').click()
    except:
        print('No cookies button so we move')
    if(main_url.split(sep='/')[4]=='42'):
        driver.find_element_by_xpath('/html/body/div[1]/main/section/div/section/nav/a[5]').click()
    else:
        driver.find_element_by_xpath('/html/body/div[1]/main/section/div/section/nav/a[4]').click()
    y=driver.current_url
    while(yes==True):
        # Enter Your own driver path below before you execute this function
        driver = webdriver.Chrome(driver_path)
        driver.get(y)
        try:
            driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div[2]/button').click()
        except:
            print('No cookies button so we move')
        driver.find_element_by_xpath('//*[@id="__next"]/main/section/div[2]/section/div/button[1]').click()
        y=driver.current_url
        driver.close()
        list1.append(y)
        if(y[-2:]=='=0'):
            yes=False
    for i in range(len(list1)):
            driver = webdriver.Chrome(driver_path)
            driver.maximize_window()
            driver.get(list1[i])
            try:
                driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div[2]/button').click()
            except:
                print('No cookies button so we move')
            time.sleep(5)
            x=driver.find_element_by_xpath('/html/body/div[1]/main/section/div[2]/section').get_attribute('outerHTML').split(sep='<')
            for i in range(len(x)):
                if(i==0):
                    continue
                if(x[i][0]=='a'):
                    list2.append(x[i].split(sep='"')[1])  
    df=pd.DataFrame()
    df['Home_Team']=[]
    df['Away_Team']=[]
    df['MatchId']=[]
    for i in range(len(list2)):
        k=[list2[i].split(sep='/')[4].split(sep='-vs-')[0],list2[i].split(sep='/')[4].split(sep='-vs-')[1],list2[i].split(sep='/')[2]]
        k1={'Home_Team':k[0],'Away_Team':k[1],'MatchId':k[2]}
        df = df.append(k1, ignore_index = True)
    df['match_urls']=list2    
    return df    

In [35]:
# Using Above Function to get Premier League Matches Data (This function notoriously takes time which I am working on)
Matches=FOTMOB_Leaguegames_Urls(Premier_League)

No cookies button so we move


In [36]:
# A DataFrame with all Matches, Id's & URL's
Matches

,Home_Team,Away_Team,MatchId,match_urls
0,afc-bournemouth,chelsea,3901272,/match/3901272/matchfacts/afc-bournemouth-vs-c...
1,manchester-city,leeds-united,3901276,/match/3901276/matchfacts/manchester-city-vs-l...
2,tottenham-hotspur,crystal-palace,3901279,/match/3901279/matchfacts/tottenham-hotspur-vs...
3,wolverhampton-wanderers,aston-villa,3901281,/match/3901281/matchfacts/wolverhampton-wander...
4,liverpool,brentford,3901275,/match/3901275/matchfacts/liverpool-vs-brentford
5,newcastle-united,arsenal,3901277,/match/3901277/matchfacts/newcastle-united-vs-...
6,west-ham-united,manchester-united,3901280,/match/3901280/matchfacts/west-ham-united-vs-m...
7,fulham,leicester-city,3901274,/match/3901274/matchfacts/fulham-vs-leicester-...
8,brighton--hove-albion,everton,3901273,/match/3901273/matchfacts/brighton--hove-albio...
9,nottingham-forest,southampton,3901278,/match/3901278/matchfacts/nottingham-forest-vs...


In [37]:
# This Function has been inspired from Chaitanya98 on Github:'https://github.com/Chaitanya98/Barcalytix/blob/main/Files%20%26%20Python%20Notebooks/FOTMOB_API.py'
import requests
def getFOTMOBShots(url, returnJSON=False):
        
    # Extracting match ID from match URL
    matchLink = url
    matchId = matchLink.split("/")[4]

    apiLink = "https://www.fotmob.com/api/matchDetails?matchId=" + matchId

    # Making request to API and loading JSON response
    response = requests.get(apiLink)
    data = json.loads(response.content)

    # Creating a pandas DataFrame from shot data    
    shotsData = pd.DataFrame(data["content"]["shotmap"]["shots"])

    # Extract the home team and away team data from the main data dictionary
    homeTeam = data['general']['homeTeam']
    awayTeam = data['general']['awayTeam']

    # Create a DataFrame to store the team names and join it with the shots data
    teamNames = pd.DataFrame([homeTeam] + [awayTeam])
    shotsData = shotsData.join(teamNames.set_index("id"), on="teamId")
    
    # Rename the `name` column to teamName
    shotsData.rename(columns={'name':'teamName'}, inplace=True)

    # Create new columns for the home team and away team names
    shotsData["homeTeam"] = homeTeam["name"]
    shotsData["awayTeam"] = awayTeam["name"]

    # Create a new column to indicate whether the shot was taken by the home or away team
    shotsData['h_a'] = np.where(shotsData["teamId"] == homeTeam["id"], 'home', 'away')

    # Update the 'eventType' column to 'Blocked' for blocked shots
    shotsData.loc[shotsData['isBlocked'], 'eventType'] = 'Blocked'

    # Calculate the distance from goal
    shotsData['distanceFromGoal'] = np.sqrt((105 - shotsData['x']) ** 2 + (34 - shotsData['y']) ** 2)
    
    # Extract the 'x' and 'y' values from the 'onGoalShot' column
    shotsData['onGoalX'] = shotsData['onGoalShot'].str['x']/2
    shotsData['onGoalY'] = shotsData['onGoalShot'].str['y']/(2/3)
    
    if returnJSON:
        return shotsData, data
    else:        
        return shotsData

In [45]:
# Get Shots data for All Brighton matches for Example (enter the team you want by checking the Matches DataFrame)
Matches=Matches[(Matches['Home_Team']=='manchester-city')|(Matches['Away_Team']=='manchester-city')]
shotsdf=pd.DataFrame()
for i in range(len(Matches)):
    df=getFOTMOBShots('https://www.fotmob.com' + Matches.iloc[i]['match_urls'])
    shotsdf=shotsdf.append(getFOTMOBShots('https://www.fotmob.com' + Matches.iloc[i]['match_urls']))    
shotsdf=shotsdf.reset_index()        
# filter the team shots off
shotsdf=shotsdf[shotsdf['teamName']=='Manchester City']

In [46]:
shotsdf

,index,id,eventType,teamId,playerId,playerName,x,y,min,minAdded,isBlocked,isOnTarget,blockedX,blockedY,goalCrossedY,goalCrossedZ,expectedGoals,expectedGoalsOnTarget,shotType,situation,period,isOwnGoal,onGoalShot,firstName,lastName,fullName,teamColor,teamName,homeTeam,awayTeam,h_a,distanceFromGoal,onGoalX,onGoalY,shortName
0,0,2544382435,Miss,8456,737066,Erling Braut Haaland,81.809223,42.466250,2,NaN,False,False,NaN,NaN,40.889762,5.549241,0.040447,NaN,LeftFoot,RegularPlay,FirstHalf,False,"{'x': 0.15914954679443183, 'y': 0.677248677248...",Erling Braut,Haaland,Erling Braut Haaland,#69A8D8,Manchester City,Manchester City,Leeds United,home,24.687841,7.957477e-02,1.015873,NaN
1,1,2544387555,Miss,8456,521318,Manuel Akanji,97.473684,35.448750,7,NaN,False,False,NaN,NaN,35.830000,6.027172,0.106609,NaN,Header,FromCorner,FirstHalf,False,"{'x': 0.7943703959313552, 'y': 0.6772486772486...",Manuel,Akanji,Manuel Akanji,#69A8D8,Manchester City,Manchester City,Leeds United,home,7.664483,3.971852e-01,1.015873,NaN
3,3,2544392811,Blocked,8456,169200,Kevin De Bruyne,84.850000,23.999236,12,NaN,True,True,95.736842,29.211429,33.695000,1.220000,0.052097,NaN,RightFoot,RegularPlay,FirstHalf,False,"{'x': 1.0806878306878307, 'y': 0.3227513211640...",Kevin,De Bruyne,Kevin De Bruyne,#69A8D8,Manchester City,Manchester City,Leeds United,home,22.495284,5.403439e-01,0.484127,NaN
4,4,2544395969,Miss,8456,974753,Julián Álvarez,95.061404,31.560000,16,NaN,False,False,NaN,NaN,30.244000,6.513793,0.166544,NaN,RightFoot,RegularPlay,FirstHalf,False,"{'x': 1.3905168596002477, 'y': 0.6772486772486...",Julián,Álvarez,Julián Álvarez,#69A8D8,Manchester City,Manchester City,Leeds United,home,10.233733,6.952584e-01,1.015873,NaN
5,5,2544396943,AttemptSaved,8456,737066,Erling Braut Haaland,94.771930,42.524062,17,NaN,False,True,103.103448,35.906250,35.753750,0.648526,0.225857,0.2806,LeftFoot,RegularPlay,FirstHalf,False,"{'x': 0.5360449735449745, 'y': 0.1715678075661...",Erling Braut,Haaland,Erling Braut Haaland,#69A8D8,Manchester City,Manchester City,Leeds United,home,13.314393,2.680225e-01,0.257352,NaN
6,6,2544399053,Goal,8456,178818,Ilkay Gündogan,89.700000,29.463571,19,NaN,False,True,NaN,NaN,30.568750,0.205474,0.108315,0.4498,RightFoot,RegularPlay,FirstHalf,False,"{'x': 1.9077380952380967, 'y': 0.0543581172486...",Ilkay,Gündogan,Ilkay Gündogan,#69A8D8,Manchester City,Manchester City,Leeds United,home,15.958358,9.538690e-01,0.081537,NaN
7,7,2544403637,Miss,8456,737066,Erling Braut Haaland,92.200000,35.448750,25,NaN,False,False,NaN,NaN,29.547619,0.179789,0.081661,NaN,LeftFoot,RegularPlay,FirstHalf,False,"{'x': 2, 'y': 0.04038052328798998, 'zoomRatio'...",Erling Braut,Haaland,Erling Braut Haaland,#69A8D8,Manchester City,Manchester City,Leeds United,home,12.881726,1.000000e+00,0.060571,NaN
8,8,2544405423,Goal,8456,178818,Ilkay Gündogan,86.997059,31.407500,27,NaN,False,True,NaN,NaN,37.126250,0.244000,0.085458,0.4317,RightFoot,RegularPlay,FirstHalf,False,"{'x': 0.1729497354497358, 'y': 0.0645502642328...",Ilkay,Gündogan,Ilkay Gündogan,#69A8D8,Manchester City,Manchester City,Leeds United,home,18.188649,8.647487e-02,0.096825,NaN
10,10,2544408903,AttemptSaved,8456,974753,Julián Álvarez,86.245588,32.551250,32,NaN,False,True,93.200000,32.932500,33.008750,0.366000,0.040694,0.0473,RightFoot,RegularPlay,FirstHalf,False,"{'x': 1.26223544973545, 'y': 0.096825396349206...",Julián,Álvarez,Julián Álvarez,#69A8D8,Manchester City,Manchester City,Leeds United,home,18.810285,6.311177e-01,0.145238,NaN
11,11,2544411271,Miss,8456,737066,Erling Braut Haaland,100.637931,22.527898,35,NaN,False,False,NaN,NaN,23.789045,0.269684,0.077871,NaN,RightFoot,RegularPlay,FirstHalf,False,"{'x': 2, 'y': 0.026411261080597232, 'zoomRatio...",Erling Braut,Haaland,Erling Braut Haaland,#69A8D8,Manchester City,Manchester City,Leeds United,home,12.273417,1.000000e+00,0.039617,NaN
